# High Value Customer Identification (Insiders)

# <font color='blue'>0.0. Imports</font>

In [41]:
import re
import numpy    as np
import pandas   as pd
import seaborn  as sns
import inflection
import warnings
import pickle
import s3fs
import os

import umap.umap_ as umap

from matplotlib import pyplot as plt
from plotly import express as px

from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import ensemble as en
from sklearn import mixture as mx
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

from scipy.cluster import hierarchy as hc

warnings.filterwarnings( 'ignore' )

In [ ]:
AWS_ACCESS_KEY_ID = os.environ.get( AWS_ACCESS_KEY_ID )
AWS_SECRET_ACCESS_KEY = os.environ.get( AWS_SECRET_ACCESS_KEY )

## <font color='blue'>0.2. Load DataSet</font>

In [2]:
# load data
df_raw = pd.read_csv('../data/raw/Ecommerce.csv', encoding='unicode_escape')

# drop extra column
df_raw = df_raw.drop( columns=['Unnamed: 8'], axis=1)

# <font color='red'>1.0. Descrição dos dados</font>

In [3]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [4]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x )

cols_new = list( map (snakecase, cols_old) )

#Rename
df1.columns = cols_new

## 1.2. Data Dimensions

In [5]:
print(f'Numero de Linhas: {df1.shape[0]}')
print(f'Numero de Colunas: {df1.shape[1]}')

Numero de Linhas: 541909
Numero de Colunas: 8


## 1.3. Data Types

In [6]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [8]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [9]:
# Criar id's acima do valor maximo do customer id, para conseguir usar os 25% da base que está como NA na clusterização

# create reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange(19000, 19000 + len( df_backup ), 1 )

# merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on='invoice_no', how='left')

# coalese
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )

# drop extra columns
df1 = df1.drop( columns=['customer_id_x', 'customer_id_y'], axis=1 )

In [10]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6. Change Dtypes

In [11]:
# invoice_date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y' )

# customer_id
df1['customer_id'] =df1['customer_id'].astype( int ) 

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


In [12]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7. Descriptive Statistics

In [13]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )


### 1.7.1 Numerical Attributes

In [14]:
# central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# dispersion - desvio padrão, minímo, maximo, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply( np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min()  ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# concatenate
metricas = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
metricas.columns=['attributes','Min', 'Max', 'Range', 'Mean', 'Mediana', 'Desvio Padrão', 'Skew', 'Kurtosis']
metricas

,attributes,Min,Max,Range,Mean,Mediana,Desvio Padrão,Skew,Kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2 Categorical Attributes

#### Invoice No

In [15]:
# prblema: temos invoice com letras e números

# identificação:
df_letter_invoce = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x) ) ), : ]
df_letter_invoce.head()

# qt invoice com letras
print(f'Total de invoice com letras: {len(df_letter_invoce)}')

# qt de linhas com a coluna 'quantity' negativa
print(f'Total de linhas com a quantity negativa: {len( df_letter_invoce[df_letter_invoce["quantity"] < 0 ] )}')

Total de invoice com letras: 9291
Total de linhas com a quantity negativa: 9288


#### Stock Code

In [16]:
# prblema: temos Stock Code com letras e números

# identificação:tock_code - apenas strings
df1.loc[df1['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$', x) ) ), 'stock_code'].unique()

# Remover stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

#### Description

In [17]:
df1.head()

# Deletar a coluna description

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


#### Country

In [18]:
# Paises únicos na base
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [19]:
# % de paises na base
df1['country'].value_counts(normalize=True).head()

country
United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: proportion, dtype: float64

In [20]:
# Qt customer por pais
df1[['customer_id', 'country']].drop_duplicates().groupby( 'country' ).count().reset_index().sort_values( 'customer_id', ascending=False ).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# <font color='red'>2.0. Filtragem de Variáveis</font>

In [21]:
df2 = df1.copy()

In [22]:
df2.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

In [23]:
# ==== Numerical Attributes ====
# unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.04, :]

# === Categorical Attributes ====
# stock code != ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] )]

# description
df2 = df2.drop( columns='description', axis=1 )

# country - exclusão das linhas (RISCO)
df2 = df2 [~df2['country'].isin ( ['European Community', 'Unspecified' ] ) ]

#bad users
df2 = df2[~df2['customer_id'].isin([16446]) ]


# quantity - Negative number (devolução de produto) - Efetuado a separação do dataframe entre os produtos vendidos e os devolvidos
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] > 0, :]


# <font color='red'>3.0. Feature Engineering</font>

In [24]:
df3 = df2.copy()

## 3.1. Feature Creation

In [25]:
# data reference
df_ref = df3.drop( ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 
                    'country'], axis=1 ).drop_duplicates( ignore_index=True)

### 3.1.1. Gross Revenue

In [26]:
# Gross Revenue (Faturamento) quantity * price
df2_purchase['gross_revenue'] = df2_purchase['quantity'] * df2_purchase['unit_price']

# Monetary
df_monetary = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2. Recency - Day from last purchase

In [27]:
# Recency - ultimo dia de compra
df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = ( df2_purchase['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.3. Quantity of purchased

In [28]:
# Numero de produtos
df_freq = (df2_purchase[['customer_id', 'invoice_no']].drop_duplicates()
                                                      .groupby('customer_id')
                                                      .count()
                                                      .reset_index()
                                                      .rename( columns={'invoice_no': 'qte_invoices'}))
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
dtype: int64

### 3.1.4. Quantity of items purchased

In [29]:
# Numero de produtos
df_freq_prod = (df2_purchase[['customer_id', 'quantity']].groupby('customer_id')
                                                         .sum()
                                                         .reset_index()
                                                         .rename( columns={'quantity': 'qte_items'}) )
df_ref = pd.merge( df_ref, df_freq_prod, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
qte_items        91
dtype: int64

### 3.1.5. Quantity of produtcs purchased

In [30]:
# Numero de produtos
df_freq_prod = (df2_purchase[['customer_id', 'stock_code']].groupby('customer_id')
                                                           .count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qtde_produtcs'}) )
df_ref = pd.merge( df_ref, df_freq_prod, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
qte_items        91
qtde_produtcs    91
dtype: int64

### 3.1.6. Average Ticket Value

In [31]:
# AVG Ticket
df_avg_ticket = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename( columns={'gross_revenue': 'avg_ticket'})
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
qte_items        91
qtde_produtcs    91
avg_ticket       91
dtype: int64

### 3.1.8. Frequency Purchase

In [32]:
df_aux = ( df2_purchase[['invoice_no', 'customer_id', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ('invoice_date', 'max'),
                                                                   min_ = ('invoice_date', 'min'),
                                                                   days_= ('invoice_date', lambda x: ( (x.max() - x.min() ).days ) +1 ),
                                                                   buy_ = ( 'invoice_no', 'count')) ).reset_index()
         
# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
qte_items        91
qtde_produtcs    91
avg_ticket       91
frequency        91
dtype: int64

### 3.1.9. Devoluções

In [33]:
# Number of Returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qte_invoices     91
qte_items        91
qtde_produtcs    91
avg_ticket       91
frequency        91
qtde_returns      0
dtype: int64

### 3.1.10. Basket Size - Quantidade de Items por Cesta (Quantity)

In [34]:
df_aux = ( df2_purchase.loc[:,['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id')
                                                                        .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                              n_products=( 'quantity', 'sum'))
                                                                        .reset_index() )

# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_basket_size']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id         0
gross_revenue      91
recency_days       91
qte_invoices       91
qte_items          91
qtde_produtcs      91
avg_ticket         91
frequency          91
qtde_returns        0
avg_basket_size    91
dtype: int64

### 3.1.11. Unique Basket Size - Quantidade de Produtos distintos por compra

In [35]:
df_aux = ( df2_purchase.loc[:,['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id')
                                                                          .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                n_products=( 'stock_code', 'nunique'))
                                                                          .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id                0
gross_revenue             91
recency_days              91
qte_invoices              91
qte_items                 91
qtde_produtcs             91
avg_ticket                91
frequency                 91
qtde_returns               0
avg_basket_size           91
avg_unique_basket_size    91
dtype: int64

# <font color='red'>4.0. EDA (Exploratory Data Analysis)</font>

In [36]:
df4 = df_ref.dropna()

## 4.3. Estudo do Espaço

In [37]:
# Selected Dataset - (Ciclo 06)
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_produtcs', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].copy()

In [42]:
mm = pp.MinMaxScaler()
fs = s3fs.S3FileSystem( anon=False, key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY )

df43['gross_revenue'] = mm.fit_transform( df43[['gross_revenue']] )
pickle.dump( mm, open( '../src/features/gross_revenue_scaler.pkl', 'wb'))
pickle.dump( mm, fs.open( 's3://insiders-dataset-tuk/gross_revenue_scaler.pkl', 'wb' ))

df43['recency_days'] = mm.fit_transform( df43[['recency_days']] )
pickle.dump( mm, open( '../src/features/recency_days_scaler.pkl', 'wb'))
pickle.dump( mm, fs.open( 's3://insiders-dataset-tuk/recency_days_scaler.pkl', 'wb' ))

df43['qtde_produtcs'] = mm.fit_transform( df43[['qtde_produtcs']] )
pickle.dump( mm, open( '../src/features/qtde_produtcs_scaler.pkl', 'wb'))
pickle.dump( mm, fs.open( 's3://insiders-dataset-tuk/qtde_produtcs_scaler.pkl', 'wb' ))

df43['frequency'] = mm.fit_transform( df43[['frequency']] )
pickle.dump( mm, open( '../src/features/frequency_scaler.pkl', 'wb'))
pickle.dump( mm, fs.open( 's3://insiders-dataset-tuk/frequency_scaler.pkl', 'wb' ))

df43['qtde_returns'] = mm.fit_transform( df43[['qtde_returns']] )
pickle.dump( mm, open( '../src/features/qtde_returns_scaler.pkl', 'wb'))
pickle.dump( mm, fs.open( 's3://insiders-dataset-tuk/qtde_returns_scaler.pkl', 'wb' ))


NameError: name 'AWS_ACCESS_KEY_ID' is not defined

### 4.3.4. Tree-Based Embedding

In [94]:
# training dataset
X = df43.drop( columns=['customer_id', 'gross_revenue'], axis=1)
y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators=300, random_state=42)

# model training
rf_model.fit( X, y )

pickle.dump( rf_model, open( '../src/models/rf_model.pkl', 'wb'))
pickle.dump( rf_model, fs.open( 's3://insiders-dataset-tuk/rf_model.pkl', 'wb' ))

# Leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

# dataframe Leaf

In [96]:
# reduzindo a dimensionalidade
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

pickle.dump( reducer, open( '../src/features/umap_reducer.pkl', 'wb'))
pickle.dump( reducer, fs.open( 's3://insiders-dataset-tuk/umap_reducer.pkl', 'wb' ))

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# <font color='red'>7.0. Hyperparameter Fine-Tunning</font>

In [42]:
X = df_tree.copy()

# <font color='red'>8.0. Model Training</font>

## 8.2. GMM

In [43]:
k = 8
# model definition
gmm_model = mx.GaussianMixture( n_components=k, n_init=300, random_state=42 )

# model training
gmm_model.fit( X )

# model predict
labels = gmm_model.predict( X )

### 8.1.0. Cluster Validation

In [44]:
print( f'SS value: {m.silhouette_score( X, labels, metric="euclidean" )}' )

SS value: 0.5811989307403564


# <font color='red'>9.0. Cluster Analysis</font>

In [55]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels
df92.head()

df92['recency_days'] = df92['recency_days'].astype( int )
df92['qtde_produtcs'] = df92['qtde_produtcs'].astype( int )
df92['qtde_returns'] = df92['qtde_returns'].astype( int )

In [56]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100 * ( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# AVG gross_revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby ('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# AVG recency_days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby ('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster')

# AVG qtde_produtcs
df_qtde_produtcs = df92[['qtde_produtcs', 'cluster']].groupby ('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_produtcs, how='inner', on='cluster')

# AVG frequency
df_frequency = df92[['frequency', 'cluster']].groupby ('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how='inner', on='cluster')

# AVG qtde_returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby ('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_returns, how='inner', on='cluster')

df_cluster.sort_values(by='gross_revenue', ascending=False)


,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtde_produtcs,frequency,qtde_returns
7,7,37,1.334295,65570.565405,16.108108,1118.216216,0.138570,1221.054054
6,6,250,9.015507,7811.725040,20.776000,338.240000,0.065782,81.432000
3,3,177,6.382979,3905.311808,27.050847,230.627119,0.036717,35.966102
0,0,361,13.018392,2603.055734,39.396122,163.249307,0.078228,23.152355
5,5,431,15.542734,1769.454269,36.143852,118.930394,0.032457,19.287703
2,2,202,7.284529,1266.267327,61.599010,85.272277,0.035871,7.698020
4,4,548,19.761991,898.286715,68.770073,64.804745,0.041638,8.202555
1,1,767,27.659574,480.256649,86.804433,39.327249,0.048856,3.074316


In [57]:
# 7 Cluster Insiders
# 6 Cluster 2 melhor
# 3 Cluster 3 melhor
# 0 Cluster 4 melhor
# 5 Cluster 5 melhor
# 2 Cluster 6 melhor
# 4 Cluster 7 melhor
# 1 Cluster pior

### Cluster 01:
    - Número de customers: 464 (16% do custormers)
    - Faturamento médio: $10.927,33
    - Recência média: 23 dias
    - Média de produtos comprados: 359 produtos
    - Frequência de produtos comprados: 0.06 compras/dia
    - Devoluções em média: 155 produtos

### Cluster 02:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 03:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 04:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 05:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 06:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 07:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

### Cluster 08:
    - Número de customers: xxx (xx% do custormers)
    - Faturamento médio: $xxxxx
    - Recência média: xx dias
    - Frequência de produtos comprados: x.xx compras/dia
    - Devoluções em média: xxx produtos

# <font color='red'>11.0. Deploy To Production</font>

In [58]:
df92.head()

,customer_id,gross_revenue,recency_days,qtde_produtcs,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6705.38,2,232,0.040323,50,6
3,13748,948.25,95,28,0.017921,0,4
4,15100,876.00,333,3,0.073171,22,1


In [59]:
df92.dtypes

customer_id        int64
gross_revenue    float64
recency_days       int64
qtde_produtcs      int64
frequency        float64
qtde_returns       int64
cluster            int64
dtype: object

## 11.1. Insert into SQLITE

In [67]:
import sqlite3
from sqlalchemy import create_engine

In [70]:
## create table
#query_create_table_insiders = '''
#    CREATE TABLE insiders(
#        customer_id      INTEGER,
#        gross_revenue    REAL,
#        recency_days     INTEGER,
#        qtde_produtcs    INTEGER,
#        frequency        REAL,
#        qtde_returns     INTEGER,
#        cluster          INTEGER        
#    )
#'''
#
#conn = sqlite3.connect( 'insiders_db.sqlite' )
#conn.execute( query_create_table_insiders )
#conn.commit()
#conn.close()

# insert data
conn = create_engine( 'sqlite:///insiders_db.sqlite' )
#df92.to_sql( 'insiders', con=conn, if_exists='append', index=False )

# select data

In [71]:
# consulta no banco de dados
query = '''
    SELECT * FROM insiders
'''

df = pd.read_sql_query( query, conn)

In [72]:
df.head()

,customer_id,gross_revenue,recency_days,qtde_produtcs,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6705.38,2,232,0.040323,50,6
3,13748,948.25,95,28,0.017921,0,4
4,15100,876.00,333,3,0.073171,22,1


In [73]:
df.shape

(2773, 7)